# Exemples de classification avec scikit-learn

On va effectuer un apprentissage supervisé multi-classes avec différents algorithmes de la librairie **scikit-learn** :
- Algorithme par analogie : K plus proches voisins
- Algorithmes par combinaison de tests élémentaires : Arbres de décision, Forêts d'arbres décisionnels
- Algorithmes par approche probabiliste : Classification naïve bayésienne
- Algorithme par maximisation de la marge : Machine à vecteurs de support
- Algorithmes par minimisation de l'erreur : Régression logistique, Algorithme du gradient stochastique, Réseau de neuronnes

Avec **scikit-learn**, le principe est toujours le même :
- Matrice de données *X* et vecteur d'étiquettes *y*
- Séparation des bases d'apprentissage et de test : *X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, ...)*
- Instanciation d'un algorithme de classification : *algo = XClassifier(...)*
- Apprentissage sur la base d'apprentissage : *algo.fit(X_train, y_train)*
- Prédiction sur la base de test : *y_pred = algo.predict(X_test)*
- Calcul de la performance en comparant y_pred avec y_test : *accuracy_score(y_test, y_pred)*

On effectuera également un apprentissage avec la librairie spécialisée de réseaux de neuronnes **Keras** qui nécessite l'installation de librairies supplémentaires : theano, mingw, libpython (sous Windows).

**scikit-learn** : Scikit-learn est une bibliothèque libre Python dédiée à l'apprentissage automatique. http://scikit-learn.org

**Keras** : Keras is a high-level neural networks library, written in Python and capable of running on top of either TensorFlow or Theano. https://keras.io

**Theano** : Theano is a Python library that allows you to define, optimize, and evaluate mathematical expressions involving multi-dimensional arrays efficiently. http://deeplearning.net/software/theano/

In [ ]:
# imports usuels
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt
%matplotlib inline

# chargement des algorithmes scikit-learn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

# utilitaires scikit-learn
from sklearn.datasets import fetch_mldata
from sklearn import model_selection
from sklearn.metrics import accuracy_score

# librairie Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

# timer
import timeit

## Chargement des données

Cet exemple utilise un dataset du MNIST voir : http://yann.lecun.com/exdb/mnist/

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

In [ ]:
from IPython.display import IFrame
IFrame("http://yann.lecun.com/exdb/mnist/", 800, 400)

In [ ]:
# import du dataset de 70.000 images en niveau de gris
# écriture manuscripte des chiffres de 0 à 9
# images étiquettées
mnist = fetch_mldata("MNIST original")

# rescale the data
X, y = mnist.data / 255., mnist.target

print(X.shape, y.shape)

In [ ]:
# images carrées de 28 x 28 pixels
np.sqrt(784)

## Affichage des données

In [ ]:
# adapté du MOOC machine learning Coursera Octave displayData.m
# et étendu avec l'affichage des chiffres en parallèle des images
# et également des écarts des prédictions

def display_data(X, y=None, y_predict=None):
    example_width = int(np.sqrt(X.shape[1]))

    # Compute rows, cols
    m, n = X.shape
    example_height = int(n / example_width)

    # Compute number of items to display
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    # Between images padding
    pad = 1

    # Setup blank display
    display_array = - np.ones((pad + display_rows * (example_height + pad),\
                           pad + display_cols * (example_width + pad)))

    # Copy each example into a patch on the display array
    # dataset contains N pixel by N pixel grayscale images of the digit 
    curr_ex = 0
    for j in range(display_rows):
        for i in range(display_cols):
            if curr_ex >= m:
                break
            # Get the max value of the patch
            max_val = max(abs(X[curr_ex]))
            display_array[(pad + j * (example_height + pad)):(pad + j * (example_height + pad))+example_height,\
                          (pad + i * (example_width + pad)):(pad + i * (example_width + pad))+example_width] =\
                            np.reshape(X[curr_ex], (example_height, example_width)) / max_val
            curr_ex += 1
        if curr_ex >= m:
            break 

    plt.figure()
    plt.axis('off')
    plt.imshow(display_array, cmap="Greys_r")
    
    if y is not None: # étiquettes des données en entrée
        df = pnd.DataFrame(y.reshape((display_rows, display_cols)))
        df = df.astype(int)
        print(df.to_string(index=False, header=False))
        
    if y_predict is not None: # étiquettes des données en sortie (écart / entrée)
        print('-'*(display_cols*3-2))
        z = pnd.Series(y).combine(pnd.Series(y_predict), func=lambda x, y: y if y != x else -1)
        df = pnd.DataFrame(z.reshape((display_rows, display_cols)))
        df = df.astype(int, raise_on_error=False)
        df.replace(to_replace=-1, value='.', inplace=True)
        print(df.to_string(index=False, header=False))

In [ ]:
# affichage aléatoire de 100 images et des chiffres correspondants
rnd = np.random.permutation(X.shape[0])[0:100]
sel = X[rnd,:]
res = y[rnd]
display_data(sel, res)

## Préparation du dataset : training set / test set

In [ ]:
# train/test split
# pour des raisons d'efficacité, on effectue l'apprentissage sur 10000 échantillons seulement
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=10000,random_state=0)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=60000,random_state=0)

## Fonction d'exécution générale

In [ ]:
# fonction d'exécution générale

algos = [] # liste des algorithmes employés
df = pnd.DataFrame() # tableau des résultats

def run(algo):
    global algos, df
    
    start = timeit.default_timer() # start chrono

    algo.fit(X_train, y_train) # algorithme d'apprentissage automatique

    y_pred = algo.predict(X_train) # prédictions sur le train set
    acc_train = accuracy_score(y_train, y_pred)
    print("Accuracy on train set: %f" % acc_train)
    
    y_pred = algo.predict(X_test) # prédictions sur le test set
    acc_test = accuracy_score(y_test, y_pred)
    print("Accuracy on test set:  %f" % acc_test)

    stop = timeit.default_timer() # stop chrono

    # agrégation des résultats
    algos.append(algo)
    df = df.append([[algo.__class__.__name__, stop-start, acc_train, acc_test]], ignore_index=True)

## K plus proches voisins / K Nearest Neighbors

Voir : http://scikit-learn.org/stable/modules/neighbors.html

In [ ]:
# méthode trop lente
#algo = KNeighborsClassifier(n_neighbors=10)

#run(algo)

## Arbres de décision / Decision Tree

Voir : http://scikit-learn.org/stable/modules/tree.html


In [ ]:
algo = DecisionTreeClassifier()

run(algo)

## Forêts d'arbres décisionnels / Random Forest

Voir : http://scikit-learn.org/stable/modules/ensemble.html#random-forests

In [ ]:
algo = RandomForestClassifier()

run(algo)

## Classification naïve bayésienne / Naive Bayes

Voir : http://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
algo = MultinomialNB()

run(algo)

## Machine à vecteurs de support / Support Vector Machine

Voir : http://scikit-learn.org/stable/modules/svm.html

In [ ]:
algo = LinearSVC()

run(algo)

## Régression logistique / Logistic Regression

Voir : http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

In [ ]:
algo = LogisticRegression()

run(algo)

## Algorithme du gradient stochastique / Stochastic Gradient Descent

Voir : http://scikit-learn.org/stable/modules/sgd.html

In [ ]:
algo = SGDClassifier()

run(algo)

## Réseau de neuronnes / Neural Network

Voir : http://scikit-learn.org/stable/modules/neural_networks_supervised.html

In [ ]:
algo = MLPClassifier()

run(algo)

## Synthèse des résultats

In [ ]:
df.columns = ['algo', 'time', 'train', 'test']
df.sort_values(by='test', ascending=False, inplace=True)
df

## Affichage des datasets moyens

In [ ]:
# dernier algo utilisé
algo.fit(X_train, y_train)
# prédiction sur toutes les données
y_pred = algo.predict(X)
# prédiction sur le test set
y_pred_test = algo.predict(X_test)

In [ ]:
# display mean tagged digit set
z1 = np.concatenate([np.mean(X[y == i], axis=0) for i in range(10)])
display_data(z1.reshape((10,28*28)))

In [ ]:
# display mean rightly recognized digit set
z3 = np.concatenate([np.mean(X[(y_pred == y) & (y == i)], axis=0) for i in range(10)])
display_data(z3.reshape((10,28*28)))

In [ ]:
# display mean wrongly recognized digit set
z3 = np.concatenate([np.mean(X[(y_pred != y) & (y == i)], axis=0) for i in range(10)])
display_data(z3.reshape((10,28*28)))

## Affichage de résultats sur le test set

In [ ]:
# affichage aléatoire de 100 chiffres et des tags associés
rnd = np.random.permutation(X_test.shape[0])[0:100]
sel = X_test[rnd,:] # 100 random digits
res = y_test[rnd]
pred = y_pred_test[rnd]
display_data(sel, res, pred)

## Prévisions sur une image fabriquée manuellement

In [ ]:
var = plt.imread("C:/Users/francis/Desktop/test2.bmp")
plt.axis('off')
plt.imshow(var, cmap='gray');

In [ ]:
# prédictions de chacun des algorithmes sur l'échantillon
sample = [var.flatten()/255]

for algo in algos:
    prediction = algo.predict(sample)[0]
    print(algo.__class__.__name__, prediction)

## Deep Learning avec les librairies Keras

In [ ]:
# train/test split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=10000,random_state=0)

In [ ]:
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
# modèle de base
def baseline_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
# test sur l'image
var = plt.imread("C:/Users/francis/Desktop/test2.bmp")
sample = var.reshape(1, 1, 28, 28)/255
model.predict(sample).argmax()

In [ ]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(30, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# build the model
model2 = larger_model()
# Fit the model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=200)
# Final evaluation of the model
scores = model2.evaluate(X_test, y_test)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
# test sur l'image
var = plt.imread("C:/Users/francis/Desktop/test2.bmp")
sample = var.reshape(1, 1, 28, 28)/255
model2.predict(sample).argmax()